# TODO Document

In [1]:
# CHANGE THIS PARAMETER
recommendee = "taapaye"

In [2]:
import functools
import os
import pickle

import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels.formula.api as smf
from tqdm import tqdm


@functools.wraps(smf.ols)
def lm(*args, **kwargs):
    return smf.ols(*args, **kwargs).fit()

In [3]:
outdir = f"../../data/recommendations/{recommendee}"
os.chdir(outdir)

In [4]:
anime = pd.read_csv("../../cleaned_data/anime.csv")

In [5]:
user_df = pickle.load(open("user_anime_list.pkl", "rb"))

In [6]:
related_df = pickle.load(open("related_loocv.pkl", "rb"))

In [7]:
records = []
for power in tqdm(range(1, 11)):
    rec_df = pickle.load(
            open(f"../../processed_data/mal_user_recs_pow_{power}.pkl", "rb")
        )
    user_recs = (
            user_df.set_index("anime_id")
            .merge(rec_df, left_on=["anime_id"], right_on=["source"])
            .drop("source", axis=1)
            .rename({"target": "anime_id"}, axis=1)
        )

    pred_scores = user_recs.groupby("anime_id").apply(
            lambda x: np.dot(x["score"], x["weight"]) / x["weight"].sum()
        )
    
    signal = pd.DataFrame()
    signal["related"] = related_df["delta"]
    signal["delta"] = pred_scores
    pred_df = user_df.merge(signal, on="anime_id", how="left").fillna(0)
    model = lm("score ~ delta + related", pred_df)
    rmse = np.sqrt(((pred_df['score'] - model.predict(pred_df)) ** 2).mean())
    records.append((power, model.rsquared_adj, rmse))
parameters = pd.DataFrame.from_records(records, columns = ['power','rsquared', 'rmse'])

100%|██████████| 10/10 [00:33<00:00,  3.35s/it]


In [8]:
parameters.sort_values(by='rsquared', ascending=False)

,power,rsquared,rmse
2,3,0.135202,1.219808
4,5,0.128881,1.224258
1,2,0.127222,1.225424
3,4,0.125609,1.226555
7,8,0.124026,1.229438
8,9,0.124026,1.229438
9,10,0.124026,1.229438
0,1,0.123610,1.227957
5,6,0.122843,1.228494
6,7,0.121947,1.229122


In [9]:
outdir = "parameters"
if not os.path.exists(outdir):
    os.mkdir(outdir)
os.chdir(outdir)

In [10]:
parameters.to_pickle("maluserrec.all.pkl")

In [11]:
parameters.sort_values(by='rsquared', ascending=False).head(1).to_pickle("maluserrec.best.pkl")

### 